<a href="https://colab.research.google.com/github/shreeshaa/BDA-homework-1/blob/master/WSCNN4150.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor
import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from copy import deepcopy
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer 

import preprocessor as p

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv("/content/drive/My Drive/WebProject/labeledTrainData.csv", delimiter="\t",names=["tweet", "label", "labelValue"])

data["label"].replace({"spam": "neutral", "normal": "neutral"}, inplace=True)
data["label"].replace({"neutral":1, "abusive": 2,"hateful":3}, inplace=True)

def preprocess_tweet(tweet):

    # 1. Remove URLs, emojis, mentions, smileys using tweet-preprocessor library
    # should not remove hashtags as hashtags contains important tweet content.
    
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION,p.OPT.RESERVED,	p.OPT.SMILEY,p.OPT.NUMBER)
    clean = p.clean(tweet)
    # 2. Remove HTML tags using Beautiful soup library
    no_tag = BeautifulSoup(clean).get_text()

    # 3. Expanding hashtags.
    
    #tweets_expanded = expand_hashtags(no_tag)

    # 4. Remove non letter char using re
    letters_only = re.sub("[^a-zA-Z]"," ", no_tag)
    #letters_only = re.sub("[^a-zA-Z]"," ", tweets_expanded)
    
    # 5. Convert to lower case
    lower_case = letters_only.lower()
    words = lower_case.split()

    new_words = []
    # ps = PorterStemmer() 
    # for w in words:
    #     new_words.append(ps.stem(w))
    for w in words:
        new_words.append(w)
    # print(new_words[0:4])
    # 5. Remove stop words
    # stops = set(stopwords.words("english")) 
    # final_words = [w for w in new_words if not w in stops]
    final_words = new_words

    return(" ".join( final_words ))

data["tweet"][0]

num_tweets = data["tweet"].size
# print(num_tweets)
for i in range(num_tweets):
    data["tweet"][i] = preprocess_tweet(data["tweet"][i])

data

hateful = data["label"]==3
hateful = data[hateful]
# hateful = hateful.sample(frac=1)
# hateful = hateful.reset_index(drop=True)
# hateful = hateful[:4333]
hateful['hateful'] = 1
hateful['abusive'] = 0
hateful['neutral'] = 0

hateful

abusive = data["label"]==2
abusive = data[abusive]
# abusive = abusive.sample(frac=1)
# abusive = abusive.reset_index(drop=True)
# abusive = abusive[:4333]
abusive['hateful'] = 0
abusive['abusive'] = 1
abusive['neutral'] = 0

abusive

neutral = data["label"]==1
neutral = data[neutral]
# neutral = neutral.sample(frac=1)
# neutral = neutral.reset_index(drop=True)
# neutral = neutral[:4334]
neutral['hateful'] = 0
neutral['abusive'] = 0
neutral['neutral'] = 1

union = pd.concat([hateful, abusive,neutral])

union = union.sample(frac=1, random_state=1).reset_index(drop=True)

union.shape

union

train = union[:69996]
validation = union[69996:79996]
test = union[79996:99996]

train[train["label"] == 3]

%matplotlib inline

!pip install transformers

import logging
import time
from platform import python_version

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from sklearn.metrics import roc_auc_score
from torch.autograd import Variable

model_class = transformers.BertModel
tokenizer_class = transformers.BertTokenizer
pretrained_weights='bert-base-uncased'
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
bert_model = model_class.from_pretrained(pretrained_weights)

max_seq = 100
target_columns = ["hateful", "abusive", "neutral"]

def tokenize_text(df, max_seq):
    return [
        tokenizer.encode(text, add_special_tokens=True)[:max_seq] for text in df.tweet.values
    ]


def pad_text(tokenized_text, max_seq):
    return np.array([el + [0] * (max_seq - len(el)) for el in tokenized_text])


def tokenize_and_pad_text(df, max_seq):
    tokenized_text = tokenize_text(df, max_seq)
    padded_text = pad_text(tokenized_text, max_seq)
    return torch.tensor(padded_text)


def targets_to_tensor(df):
    return torch.tensor(df[target_columns].values, dtype=torch.float32)

train_indices = tokenize_and_pad_text(train, max_seq)
val_indices = tokenize_and_pad_text(validation, max_seq)
test_indices = tokenize_and_pad_text(test, max_seq)

train_indices.shape

with torch.no_grad():
  for i in range(10,20):
    if i == 10:
      x_train = bert_model(train_indices[:1000])[0]
      x_val = bert_model(val_indices[:1000])[0]
      x_test = bert_model(test_indices[:1000])[0]
    else:
      x_train = torch.cat((x_train,bert_model(train_indices[i*1000:(i+1)*1000])[0]),dim=0)
      x_val = torch.cat((x_val,bert_model(val_indices[i*1000:(i+1)*1000])[0]),dim=0)
      x_test = torch.cat((x_test,bert_model(test_indices[i*1000:(i+1)*1000])[0]),dim=0)

x_train

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=d9aa3e3ef2cde5e94f50cee6baf129e2516be4aeb7745d338062127a29bbfac6
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 11.0MB/s 
     |████████████████████████████████| 1.0MB 41.0MB/s 
     |████████████████████████████████| 890kB 48.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=85958e5fdffeb527427911fe447dae854b6b6b6b972a086962bcb0912bf53493
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


tensor([[[-0.4374,  0.3052,  0.4779,  ..., -0.2994,  0.4771, -0.5297],
         [-0.3174,  0.2385,  0.3197,  ..., -0.4987,  0.4756,  0.2309],
         [-0.1331,  0.2687,  1.1933,  ..., -0.3334,  0.5221,  0.4827],
         ...,
         [ 0.2037, -0.1072,  0.9296,  ..., -0.5398,  0.2554, -0.8074],
         [ 0.1017, -0.1093,  0.9060,  ..., -0.5195,  0.1509, -0.7584],
         [ 0.1147,  0.0751,  0.8272,  ..., -0.5050,  0.1400, -0.8131]],

        [[-0.5415,  0.3170,  0.4304,  ..., -0.4812,  0.4469, -0.4659],
         [-0.2192,  0.6637, -0.0265,  ..., -0.6692,  0.1630, -0.3818],
         [-0.4265,  0.4721,  0.2064,  ..., -0.3100,  0.0573,  0.3148],
         ...,
         [ 0.0918, -0.2180,  0.9180,  ..., -0.5563,  0.2320, -0.7675],
         [-0.0831, -0.1838,  0.9331,  ..., -0.5530,  0.1662, -0.6904],
         [-0.0162,  0.0234,  0.8659,  ..., -0.6017,  0.1680, -0.8301]],

        [[-0.4318,  0.4026,  0.4854,  ..., -0.3491,  0.5271, -0.5924],
         [ 0.1241,  1.6540,  0.7956,  ...,  0

In [0]:
torch.save(x_train,'xtrain1020.pt')
torch.save(x_val,'xval1020.pt')
torch.save(x_test,'xtest1020.pt')

In [13]:
print("Hi")

Hi


In [14]:
with torch.no_grad():
  for i in range(40,50):
    if i == 40:
      x_train = bert_model(train_indices[:1000])[0]
      x_val = bert_model(val_indices[:1000])[0]
      x_test = bert_model(test_indices[:1000])[0]
    else:
      x_train = torch.cat((x_train,bert_model(train_indices[i*1000:(i+1)*1000])[0]),dim=0)
      x_val = torch.cat((x_val,bert_model(val_indices[i*1000:(i+1)*1000])[0]),dim=0)
      x_test = torch.cat((x_test,bert_model(test_indices[i*1000:(i+1)*1000])[0]),dim=0)

x_train

tensor([[[-0.4374,  0.3052,  0.4779,  ..., -0.2994,  0.4771, -0.5297],
         [-0.3174,  0.2385,  0.3197,  ..., -0.4987,  0.4756,  0.2309],
         [-0.1331,  0.2687,  1.1933,  ..., -0.3334,  0.5221,  0.4827],
         ...,
         [ 0.2037, -0.1072,  0.9296,  ..., -0.5398,  0.2554, -0.8074],
         [ 0.1017, -0.1093,  0.9060,  ..., -0.5195,  0.1509, -0.7584],
         [ 0.1147,  0.0751,  0.8272,  ..., -0.5050,  0.1400, -0.8131]],

        [[-0.5415,  0.3170,  0.4304,  ..., -0.4812,  0.4469, -0.4659],
         [-0.2192,  0.6637, -0.0265,  ..., -0.6692,  0.1630, -0.3818],
         [-0.4265,  0.4721,  0.2064,  ..., -0.3100,  0.0573,  0.3148],
         ...,
         [ 0.0918, -0.2180,  0.9180,  ..., -0.5563,  0.2320, -0.7675],
         [-0.0831, -0.1838,  0.9331,  ..., -0.5530,  0.1662, -0.6904],
         [-0.0162,  0.0234,  0.8659,  ..., -0.6017,  0.1680, -0.8301]],

        [[-0.4318,  0.4026,  0.4854,  ..., -0.3491,  0.5271, -0.5924],
         [ 0.1241,  1.6540,  0.7956,  ...,  0

In [17]:
x_test

tensor([[[-4.6289e-01,  2.7931e-01,  5.0270e-01,  ..., -3.6187e-01,
           5.1460e-01, -4.9478e-01],
         [ 6.5242e-02, -7.6948e-02,  2.3396e-01,  ...,  2.6637e-01,
           7.0237e-01,  1.1103e+00],
         [-5.0864e-01, -3.6947e-01,  5.5570e-01,  ..., -1.6562e-01,
          -2.8545e-01, -3.9330e-01],
         ...,
         [ 2.2954e-01, -1.2384e-01,  9.3870e-01,  ..., -5.9352e-01,
           2.1446e-01, -9.5107e-01],
         [ 1.4961e-01, -1.1893e-01,  9.1185e-01,  ..., -5.7282e-01,
           1.3025e-01, -8.7696e-01],
         [ 1.4844e-01,  3.1131e-02,  8.2199e-01,  ..., -5.4887e-01,
           1.1726e-01, -9.1609e-01]],

        [[-5.2729e-01,  3.5245e-01,  4.7916e-01,  ..., -4.5925e-01,
           3.7977e-01, -4.0403e-01],
         [ 2.3086e-01,  1.2603e-01,  6.7737e-02,  ..., -2.0803e-01,
           4.2145e-01, -1.6338e-01],
         [ 4.7519e-02,  4.2805e-02,  2.0243e-01,  ..., -2.8018e-01,
          -2.6501e-01, -1.4484e-01],
         ...,
         [ 1.7404e-01, -1

In [0]:
torch.save(x_train,'xtrain4050.pt')

In [0]:
x_train_4150 = x_train[1000:,:,:].shape

In [0]:
torch.save(x_train_4150,'xtrain4150.pt')

In [0]:
x_train_4150 = torch.load('xtrain4150.pt')

In [0]:
import torch
x_train_1020 = torch.load('xtrain1020.pt')

In [0]:
x_train_4050 = torch.load('xtrain4050.pt')

In [4]:
x_train_1020.shape

torch.Size([10000, 100, 768])

In [11]:
x_train_4050.shape

torch.Size([10000, 100, 768])

In [0]:
x_train_4150 = x_train_4050[1000:,:,:]

In [14]:
x_train_4150.shape

torch.Size([9000, 100, 768])

In [0]:
x_train_1120 = x_train_1020[1000:,:,:]

In [16]:
x_train_1120.shape

torch.Size([9000, 100, 768])

In [0]:
torch.save(x_train_1120,'x_train_1120.pt')
torch.save(x_train_4150,'x_train_4150.pt')

In [0]:
x_train_11204150 = torch.cat((x_train_1120,x_train_4150),dim=0)

In [20]:
x_train_11204150.shape

torch.Size([18000, 100, 768])